<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Named Entity Recognition

We will use the Spacy Library:
https://spacy.io/usage/spacy-101


<img src="https://drive.google.com/uc?export=view&id=1m_EMdnI5C826kgqK7r5vB4TXnB0-Wq7W" alt="Intestazione con loghi istituzionali" width="525"/>

| Docente      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Matteo Lissandrini      | Laboratorio Avanzato di Informatica Umanistica       | 2023/2024   |

## Usual install and basic imports

In [22]:
%pip install wikipedia-api
%pip install spacy==3.7.0

In [40]:
import gzip
import math
import string
import requests
import numpy as np
import regex as re
import matplotlib.pyplot as plt
from collections import Counter

punct_regex = re.compile('[{}]'.format(re.escape(string.punctuation))) # Regex matching any punctuation
space_regex = re.compile(' +') # Regex matching whitespace

In [24]:
import spacy
from spacy import displacy
# Load module for english
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


## Test the NER methods

In [25]:
doc = nlp("Paris Hilton welcomes 2nd child with Carter Reum")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Paris Hilton 0 12 PERSON
2nd 22 25 ORDINAL
Carter Reum 37 48 PERSON


In [26]:
# We can render in a nice format our annotations
displacy.render(doc, style="ent", jupyter=True)

In [28]:
doc = nlp("The Hilton Paris  hotel welcomes this year more than 1640 guests")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent", jupyter=True)

Hilton Paris 4 16 GPE
this year 33 42 DATE
more than 1640 43 57 CARDINAL


In [29]:
doc = nlp("Hilton Paris: Born in New York City, and raised there and in Los Angeles ")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent", jupyter=True)

Hilton Paris 0 12 PERSON
New York City 22 35 GPE
Los Angeles 61 72 GPE


In [30]:
# Longer document
doc = nlp("""
Citing high fuel prices, United Airlines said Friday it has increased fares by $6
per round trip on flights to some cities also served by lower-cost carriers.
American Airlines, a unit of AMR Corp., immediately matched the move,
spokesman Tim Wagner said.
""")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(doc, style="ent", jupyter=True)

United Airlines 26 41 ORG
Friday 47 53 DATE
6 81 82 MONEY
American Airlines 162 179 ORG
AMR Corp. 191 200 ORG
Tim Wagner 243 253 PERSON


## Goal:  Test how NER would work on Text from Alice in Wonderland and Aesop's Fables



In [44]:
def get_pages(book_text):
  """
  Function that given  the book text returns a list of pages
  """
  _pages = [ _page.strip() for _page in book_text.split("\n\r\n\r\n\r")] # pages are divided by multiple newlines
  _pages = [ space_regex.sub(' ', page).strip() for page in _pages ]
  _pages = [ space_regex.sub(' ', " ".join(page.splitlines())) for page in _pages ]
  _pages = [ _page for _page in _pages if _pages != '' ]

  return _pages

In [45]:
# request the raw text of Alice in Wonderland
r = requests.get(r'https://ia801604.us.archive.org/6/items/alicesadventures19033gut/19033.txt')
alice = r.text

alice_pages = get_pages(alice)


r = requests.get(r'https://ia600906.us.archive.org/29/items/aesopsfablesanew11339gut/11339.txt')
fables = r.text

fables_pages = get_pages(fables)



In [51]:
test_page = alice_pages[16]
doc = nlp(test_page)
displacy.render(doc, style="ent", jupyter=True)

**If you are curious about `entity linking` you can see this tutorial:**

https://github.com/explosion/projects/blob/v3/tutorials/nel_emerson/notebooks/notebook_video.ipynb